question 1: 

Task: use 17 industry portfolio as regressor, estimate HML and MOM 

1. create HML and MOM using 6 portfolios
2. the returns of HML is factor return of value portoflio; the returns of MOM is factor return of momentum portfolio

*portfolio of each year seems to be constructed based on data at the end of June of the same year, so do the same for this?

For each of HML and MOM: 
1. regress 17 industry portfolio on HML/MOM

2. run one of each type of regression: linear (forward and hybrid), ridge, LASSO, and random forest 
2a. Hybird on MOM gave same model set as forward, and we already know the two variable one has lowest CV SSE, so two variable is the best pick for MOM Linear, so i didn't write code for hybrid again

3. pick best 2


Compare selection criteria across models and across (1a), (1b), and (1c)
1. For selection of optimal model with the same number of predictors we only use in-sample SSE
2. For overall selection we use CV SSE, AIC, and BIC (in slide)

  

select three best models for HML and for MOM separately for question 1 

  

question 2: 

Task: use 17 industry portfolio as regressor, Comment on the ease or difficulty of replicating the components as opposed to the entire return 

1. use three best model for HML and compare that with accuracy of SV+BV and SG+BG 

2. use three best model for MOM and compare that with accuracy of SH+BH and SL+BL 

  

question 3: 

Task: combine the three procedures, and test accuracy on HML and MOM vs each of the component 

if still have word count, can test accuracy on long-side and short side and/or do pairs combinations of three procedures 


In [2]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from collections import Counter
import math

In [3]:
mom = pd.read_csv("factor_return_mom.csv")

hml = pd.read_csv("factor_return_value.csv")

#The 6 Portfolios Formed on Size and Momentum (contains SH BH SL BL)
size_momentum = pd.read_csv("6_Portfolios_size_mom.csv")
size_momentum = size_momentum.iloc[:1174]
size_momentum['Unnamed: 0'] = size_momentum['Unnamed: 0'].astype(int)

#The 6 Portfolios Formed on Size and Value (contains SV BV SG BG)
size_value = pd.read_csv("6_Portfolios_size_value.csv")
#small lo seems to be small growth

#The 17 Industry Portfolios (regressors)
seventeen = pd.read_csv("17_Industry_Portfolios_cleaned.csv")

# Display the first few rows of the DataFrame
print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

   Unnamed: 0   HML
0      192607 -2.43
1      192608  3.82
2      192609  0.13
3      192610  0.70
4      192611 -0.51
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
   Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
0      192607      1.0874   0.9081     -0.0695    5.7168   1.8971    2.0066
1      192608

In [4]:
#make everything start at 192701
hml = hml.iloc[6:]
size_value = size_value.iloc[6:]
seventeen = seventeen.iloc[6:]

print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

# check if times are the same, or some dataset is missing certain dates

    Unnamed: 0   HML
6       192701  4.54
7       192702  2.94
8       192703 -2.61
9       192704  0.81
10      192705  4.73
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
    Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
6       192701     -0.7991  -0.3782      3.9316   -0.2621   0.0321    4.0885
7    

In [5]:
size_momentum['Unnamed: 0'] = pd.to_datetime(size_momentum['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
size_value['Unnamed: 0'] = pd.to_datetime(size_value['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
hml['Unnamed: 0'] = pd.to_datetime(hml['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
mom['Unnamed: 0'] = pd.to_datetime(mom['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
seventeen['Unnamed: 0'] = pd.to_datetime(seventeen['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')

In [6]:
print(seventeen.columns)
seventeen.columns = seventeen.columns.str.strip() #strip spaces
predictors = seventeen.columns[1:]

Index(['Unnamed: 0', 'Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems',
       'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils',
       'Rtail', 'Finan', 'Other'],
      dtype='object')


In [7]:
# Set date as the index
seventeen.set_index('Unnamed: 0', inplace=True)
size_momentum.set_index('Unnamed: 0', inplace=True)
size_value.set_index('Unnamed: 0', inplace=True)
hml.set_index('Unnamed: 0', inplace=True)
mom.set_index('Unnamed: 0', inplace=True)

In [8]:
#group data into 5-year intervals
seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
size_value_grouped = size_value.groupby(pd.Grouper(freq='5Y'))
mom_grouped = mom.groupby(pd.Grouper(freq='5Y'))
hml_grouped = hml.groupby(pd.Grouper(freq='5Y'))

/tmp/ipykernel_1917/903688633.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1917/903688633.py:2: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1917/903688633.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1917/903688633.py:3: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1917/903688633.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_value_grouped = si

In [9]:
group_labels = list(seventeen_grouped.groups.keys())

In [10]:
print(group_labels)

[Period('1927', '5Y-DEC'), Period('1932', '5Y-DEC'), Period('1937', '5Y-DEC'), Period('1942', '5Y-DEC'), Period('1947', '5Y-DEC'), Period('1952', '5Y-DEC'), Period('1957', '5Y-DEC'), Period('1962', '5Y-DEC'), Period('1967', '5Y-DEC'), Period('1972', '5Y-DEC'), Period('1977', '5Y-DEC'), Period('1982', '5Y-DEC'), Period('1987', '5Y-DEC'), Period('1992', '5Y-DEC'), Period('1997', '5Y-DEC'), Period('2002', '5Y-DEC'), Period('2007', '5Y-DEC'), Period('2012', '5Y-DEC'), Period('2017', '5Y-DEC'), Period('2022', '5Y-DEC')]


# Analysis (1a)

## 5-year data

### Linear regression

In [11]:
# linear (variable selection)

linear = LinearRegression(fit_intercept=False)

#### forward stepwise selection

In [12]:
###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3533222164.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


/tmp/ipykernel_1917/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3533222164.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [13]:
###### two variable

# first selected variable is Trans for both

#remove from predictors
predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors1:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


/tmp/ipykernel_1917/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


In [14]:
###### three variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)

#remove from predictors
predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Mines


/tmp/ipykernel_1917/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [15]:
###### four variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1917/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [16]:
###### five variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Rtail


In [17]:
###### six variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: FabPr


In [18]:
###### seven variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1917/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [19]:
###### eight variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Chems


In [20]:
###### nine variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Chems'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Food


/tmp/ipykernel_1917/2104319118.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [21]:
###### ten variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Food'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [22]:
###### 11 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Steel


In [23]:
###### 12 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Clths'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: Oil


/tmp/ipykernel_1917/2863303116.py:62: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [24]:
###### 13 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Cnsum


/tmp/ipykernel_1917/412403489.py:65: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [25]:
###### 14 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Cnsum'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Utils


In [26]:
###### 15 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Utils'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Food
The MOM predictor with the lowest SSE is: Cars


In [27]:
###### 16 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils + Cars (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Food'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

The HML predictor with the lowest SSE is: Mines
The MOM predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1917/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2679972848.py:74: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [28]:
# Best Models
best_model_hml = []
best_model_mom = []

# first selected variable is Trans for both
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
best_model_hml.append(['Trans','Machn'])
best_model_mom.append(['Trans','Cnstr'])

# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
best_model_hml.append(['Trans','Machn', 'Steel'])
best_model_mom.append(['Trans','Cnstr', 'Mines'])
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan'])
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail'])
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr'])
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths'])
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems'])
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food'])
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn'])
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel'])
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil'])
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum'])
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils'])
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars'])
# 16 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food + Mines (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars + Durbl (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food', 'Mines'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars', 'Durbl'])
#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [29]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
            train = explanatory.get_group(group_labels[i]).values
            test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
             train = explanatory.get_group(group_labels[i]).values
             test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude = hml_grouped.get_group(group_labels[0])
mom_to_exclude = mom_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values
mom_5yr_actual = mom[~mom.index.isin(mom_to_exclude.index)]
mom_5yr_actual = mom_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)
n_mom = len(mom_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

/tmp/ipykernel_1917/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [30]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest SSE is: ['Trans', 'Cnstr']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest AIC is: ['Trans', 'Cnstr']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest BIC is: ['Trans', 'Cnstr']


#### hybrid stepwise selection

In [31]:
### hybrid stepwise selection

# use the optimal model got from forward

# 5 to 4 

#store sse
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

mom_pred = pd.Series(['Trans', 'Cnstr'])

for pred in mom_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_mom = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")


/tmp/ipykernel_1917/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1917/2367577185.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [32]:
# 4 to 3

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [33]:
# 3 to 2

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Steel', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [34]:
# 2 to 1

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3282136415.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3282136415.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Finan


In [35]:
#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

best_model_hml = ['Trans', ['Trans', 'Finan'], ['Trans', 'Steel', 'Finan'], ['Trans', 'Machn', 'Steel', 'Finan'], 
                  ['Trans', 'Machn', 'Steel', 'Finan', 'Other']]

#store predicted value
predict_hml_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
            train = explanatory.get_group(group_labels[i]).values
            test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

hml_to_exclude = hml_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

/tmp/ipykernel_1917/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [36]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']


In [37]:
### backward stepwise selection

### Ridge

In [38]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-1):
        train = seventeen_grouped.get_group(group_labels[i]).values
        test = seventeen_grouped.get_group(group_labels[i+1]).values
        
        #estimate HML using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
        #estimate MOM using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_ridge_a = float(best_hml)
print(f"The alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_ridge_a = float(best_mom)
print(f"The alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3120643919.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3120643919.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3120643919.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The alpha with the lowest SSE is: 100.0
The alpha with the lowest SSE is: 1000.0


### LASSO

In [39]:
### lasso

# we can perhaps look at the variables chosen by LASSO if we want to talk about and iterpret them

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-1):
        train = seventeen_grouped.get_group(group_labels[i]).values
        test = seventeen_grouped.get_group(group_labels[i+1]).values
        
        #estimate HML using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
        
        #predict next 5 year HML
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
        #estimate MOM using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    
        #predict next 5 year MOM
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_lasso_a = float(best_hml)
print(f"The alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_lasso_a = float(best_mom)
print(f"The alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2867682998.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.009e+02, tolerance: 4.859e+01
  model = cd_fast.enet_coordinate_descent(
/tmp/ipykernel_1917/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the con

The alpha with the lowest SSE is: 0.31622776601683794
The alpha with the lowest SSE is: 10.0


/tmp/ipykernel_1917/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2867682998.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

### Random Forest

In [40]:
train = seventeen_grouped.get_group(group_labels[1]).values
test = seventeen_grouped.get_group(group_labels[2]).values
data = np.concatenate((train, test), axis=0)

#random forest regression

#set seed to 42
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'max_features': ['sqrt'] #slide 7 said use sqrt p predictors where p is the total number of predictors
}

# Time series cross-validator
tscv = TimeSeriesSplit(n_splits=5) #this way the cross validation only happens in chronological order

#create model
rf = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid,
                                   n_iter=100, cv=tscv, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

# tune hyperparameters

param_hml = []
param_mom = []

for i in range(0,len(group_labels)-1):
    x_train = seventeen_grouped.get_group(group_labels[i]).values
    hml_train = hml_grouped.get_group(group_labels[i]).values
    mom_train = mom_grouped.get_group(group_labels[i]).values

    x_test = seventeen_grouped.get_group(group_labels[i+1]).values
    hml_test = hml_grouped.get_group(group_labels[i+1]).values
    mom_test = mom_grouped.get_group(group_labels[i+1]).values

    x = np.concatenate((x_train, x_test), axis=0)
    h = np.concatenate((hml_train, hml_test), axis=0)
    m = np.concatenate((mom_train, mom_test), axis=0)

    rf.fit(x, h)
    param_hml.append(rf.best_params_)

    rf.fit(x, m)
    param_mom.append(rf.best_params_)

print("For HML")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_hml]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_hml]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_hml]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_hml]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

print("For MOM")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_mom]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_mom]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_mom]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_mom]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

In [41]:
#overall selection

# forward & hybird selection: The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
### The MOM predictor with the lowest SSE is: ['Trans', 'Cnstr']

#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
predict_hml = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Machn', 'Steel', 'Finan', 'Other']]
for i in range(0,len(group_labels)-1):
    train = explanatory.get_group(group_labels[i]).values
    test = explanatory.get_group(group_labels[i+1]).values

    #estimate HML using 5 year data
    linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
    #predict next 5 year HML
    predict = pd.Series(linear.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])
    
predict_hml_df['Linear'] = predict_hml

#MOM
predict_mom = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Cnstr']]
for i in range(0,len(group_labels)-1):
    train = explanatory.get_group(group_labels[i]).values
    test = explanatory.get_group(group_labels[i+1]).values

    #estimate MOM using 5 year data
    linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
    #predict next 5 year MOM
    predict = pd.Series(linear.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])
    
predict_mom_df['Linear'] = predict_mom

# ridge: HML alpha = 100 (stored in hml_ridge_a), MOM alpha = 1000 (stored in mom_ridge_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate HML using 5 year data
    ridge_model = Ridge(alpha=hml_ridge_a, fit_intercept=False)
    ridge_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
    hml_coef_ridge = ridge_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df['Ridge'] = predict_hml
    
#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate MOM using 5 year data
    ridge_model = Ridge(alpha=mom_ridge_a, fit_intercept=False)
    ridge_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    mom_coef_ridge = ridge_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df['Ridge'] = predict_mom

# lasso: HML alpha = 0.31622776601683794 (stored in hml_lasso_a), MOM alpha = 10 (stored in mom_lasso_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate HML using 5 year data
    lasso_model = Lasso(alpha=hml_lasso_a, fit_intercept=False)
    lasso_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
    hml_coef_lasso = lasso_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df['LASSO'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate MOM using 5 year data
    lasso_model = Lasso(alpha=mom_lasso_a, fit_intercept=False)
    lasso_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    mom_coef_lasso = lasso_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df['LASSO'] = predict_mom

# random forest

# pick best 2

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

/tmp/ipykernel_1917/234000951.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/234000951.py:38: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/234000951.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [42]:
# count num of non zero coef
k_ridge_hml = np.count_nonzero(hml_coef_ridge)
k_ridge_mom = np.count_nonzero(mom_coef_ridge)

k_lasso_hml = np.count_nonzero(hml_coef_lasso)
k_lasso_mom = np.count_nonzero(mom_coef_lasso)

k_l_hml = 5
k_l_mom = 2

k_hml = [k_l_hml, k_ridge_hml, k_lasso_hml]

k_mom = [k_l_mom, k_ridge_mom, k_lasso_mom]

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = sse_hml.index
bic_hml.index = sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = sse_mom.index
bic_mom.index = sse_mom.index

#sort in ascending order
sse_hml = sse_hml.sort_values()
sse_mom = sse_mom.sort_values()
aic_hml = aic_hml.sort_values()
aic_mom = aic_mom.sort_values()
bic_hml = bic_hml.sort_values()
bic_mom = bic_mom.sort_values()

In [43]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
second = sse_hml.index[1]
print(f"The HML predictor with the second lowest SSE is: {second}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")
second = sse_mom.index[1]
print(f"The MOM predictor with the second lowest SSE is: {second}")

# Choose optimal model (minimum AIC)
best_hml = aic_hml.idxmin()
print(f"The HML predictor with the lowest AIC is: {best_hml}")
second = aic_hml.index[1]
print(f"The HML predictor with the second lowest AIC is: {second}")
best_mom = aic_mom.idxmin()
print(f"The MOM predictor with the lowest AIC is: {best_mom}")
second = aic_mom.index[1]
print(f"The MOM predictor with the second lowest AIC is: {second}")

# Choose optimal model (minimum BIC)
best_hml = bic_hml.idxmin()
print(f"The HML predictor with the lowest BIC is: {best_hml}")
second = bic_hml.index[1]
print(f"The HML predictor with the second lowest BIC is: {second}")
best_mom = bic_mom.idxmin()
print(f"The MOM predictor with the lowest BIC is: {best_mom}")
second = bic_mom.index[1]
print(f"The MOM predictor with the second lowest BIC is: {second}")

The HML predictor with the lowest SSE is: Linear
The HML predictor with the second lowest SSE is: Ridge
The MOM predictor with the lowest SSE is: Ridge
The MOM predictor with the second lowest SSE is: LASSO
The HML predictor with the lowest AIC is: Linear
The HML predictor with the second lowest AIC is: LASSO
The MOM predictor with the lowest AIC is: Ridge
The MOM predictor with the second lowest AIC is: LASSO
The HML predictor with the lowest BIC is: Linear
The HML predictor with the second lowest BIC is: LASSO
The MOM predictor with the lowest BIC is: Ridge
The MOM predictor with the second lowest BIC is: LASSO


## 10-year data

### Linear Regression

In [44]:
###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
        train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3889845230.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


In [45]:
###### two variable

predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1207218035.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1207218035.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [46]:
###### 3 variable

predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Mines


/tmp/ipykernel_1917/2062258075.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [47]:
###### 4 variable

predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [48]:
###### 5 variable

predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [49]:
###### 6 variable

predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1917/1363685795.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1363685795.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1363685795.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [50]:
###### 7 variable

predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/3856651286.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [51]:
###### 8 variable

predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Cnsum


/tmp/ipykernel_1917/669673142.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/669673142.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/669673142.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [52]:
###### 9 variable

predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Cnsum'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: Oil


/tmp/ipykernel_1917/2643745856.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2643745856.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2643745856.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [53]:
###### 10 variable

predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Rtail


In [54]:
###### 11 variable

predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/2546659055.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [55]:
###### 12 variable

predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1917/2287680030.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [56]:
###### 13 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Durbl'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Food
The MOM predictor with the lowest SSE is: Chems


In [57]:
###### 14 variable

predictorsh = predictorsh[~predictorsh.isin(['Food'])]
predictorsm = predictorsm[~predictorsm.isin(['Chems'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: FabPr


/tmp/ipykernel_1917/1735760812.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1735760812.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [58]:
###### 15 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food', 'Cnstr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: FabPr
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/3876624578.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3876624578.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [59]:
###### 16 variable

predictorsh = predictorsh[~predictorsh.isin(['FabPr'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2326514894.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2326514894.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2326514894.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Utils


/tmp/ipykernel_1917/2326514894.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2326514894.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [60]:
# best models

best_model_hml = []
best_model_mom = []

# 1 var
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# 2 var
best_model_hml.append(['Trans', 'Machn'])
best_model_mom.append(['Trans', 'Cnstr'])

# 3 var
best_model_hml.append(['Trans', 'Machn', 'Steel'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines'])

# 4 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths'])

# 5 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel'])

# 6 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan'])

# 7 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other'])

# 8 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum'])

# 9 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil'])

# 10 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail'])

# 11 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars'])

# 12 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl'])

# 13 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems'])

# 14 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr'])

# 15 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn'])

# 16 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr', 'Clths'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn', 'Utils'])

#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [61]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude1 = hml_grouped.get_group(group_labels[0])
mom_to_exclude1 = mom_grouped.get_group(group_labels[0])
hml_to_exclude2 = hml_grouped.get_group(group_labels[1])
mom_to_exclude2 = mom_grouped.get_group(group_labels[1])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude1.index)]
hml_5yr_actual = hml_5yr_actual[~hml_5yr_actual.index.isin(hml_to_exclude2.index)]
hml_5yr_actual = hml_5yr_actual.values
mom_5yr_actual = mom[~mom.index.isin(mom_to_exclude1.index)]
mom_5yr_actual = mom_5yr_actual[~mom_5yr_actual.index.isin(mom_to_exclude2.index)]
mom_5yr_actual = mom_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)
n_mom = len(mom_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

/tmp/ipykernel_1917/56840858.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/56840858.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/56840858.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


In [62]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest SSE is: Trans
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest AIC is: Trans
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest BIC is: Trans


### Hybird

In [63]:
#hybrid

# can only do it on HML since MOM is only one predictor

### hybrid stepwise selection

# use the optimal model got from forward

# 6 to 5

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [64]:
# 5 to 4

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [65]:
# 4 to 3

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [66]:
# 3 to 2

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Finan


/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [67]:
# 2 to 1

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/2328425969.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2328425969.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [68]:
#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

best_model_hml = ['Trans', ['Trans', 'Other'], ['Trans', 'Other', 'Finan'], ['Trans', 'Machn', 'Finan', 'Other'], 
                  ['Trans', 'Machn', 'Steel', 'Finan', 'Other'], ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]

#store predicted value
predict_hml_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

k_values = list(range(1, 7))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

/tmp/ipykernel_1917/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [69]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']


### Ridge

In [107]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        test = explanatory.get_group(group_labels[i+2]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
        
        #estimate HML using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
    
        #estimate MOM using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_ridge_a = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_ridge_a = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1937/3059505820.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/3059505820.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1937/3059505820.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The alpha with the lowest SSE is: 100.0
The alpha with the lowest SSE is: 10000.0


### LASSO

In [109]:
# ridge and lasso (tune alpha)

### lasso

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        test = explanatory.get_group(group_labels[i+2]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
        
        #estimate HML using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
    
        #estimate MOM using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_lasso_a = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_lasso_a = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1937/1211543296.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/1211543296.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1937/1211543296.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML alpha with the lowest SSE is: 0.1
The MOM alpha with the lowest SSE is: 316.2277660168379


/tmp/ipykernel_1937/1211543296.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/1211543296.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1937/1211543296.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

### Random Forest

In [108]:
#overall

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#linear (forward & hybrid)
# The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']

predict_hml = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]
for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 5 year data
    linear.fit(train,y)
    #predict next 5 year HML
    predict = pd.Series(linear.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])
    
predict_hml_df['Linear'] = predict_hml

# The MOM predictor with the lowest SSE is: Trans
predict_mom = pd.Series()
explanatory = seventeen_grouped['Trans']
for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 5 year data
    linear.fit(train,y)
    #predict next 5 year MOM
    predict = pd.Series(linear.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])
    
predict_mom_df['Linear'] = predict_mom

#ridge
# HML alpha: 100.0 (stored in hml_ridge_a), MOM alpha: 10000.0 (stored in mom_ridge_a)
#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 5 year data
    ridge_model = Ridge(alpha=hml_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    hml_coef_ridge = ridge_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df['Ridge'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 5 year data
    ridge_model = Ridge(alpha=mom_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    mom_coef_ridge = ridge_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df['Ridge'] = predict_mom

#lasso
# HML alpha: 0.1 (stored in hml_lasso_a), MOM alpha: 316.2277660168379 (stored in mom_lasso_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 5 year data
    lasso_model = Ridge(alpha=hml_lasso_a, fit_intercept=False)
    lasso_model.fit(train, y)
    hml_coef_lasso = lasso_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df['Lasso'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 5 year data
    lasso_model = Lasso(alpha=mom_lasso_a, fit_intercept=False)
    lasso_model.fit(train, y)
    mom_coef_lasso = lasso_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df['Lasso'] = predict_mom

# random forest

# pick best 2

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

## 20-year data